In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import sosfilt, butter
from tqdm import tqdm
import os

In [5]:
# The bandpass function creates weights according to params that the filter uses.
root = '/home/karthiktiwari/Data/dataset/'

def bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], 'bandpass', analog=False, output="sos")
    return sos

cutoff_high = 30
cutoff_low = 0.5

fs = 200
order = 3
sos = bandpass(cutoff_low, cutoff_high, fs, order=order)

#sos => weights for filter. We can use this to clean the signal.
def prep(sos, signal):
    maxx = np.max(signal)
    minn = np.min(signal)
    signal = (signal - minn) / (maxx - minn + 1e-6)
    signal = sosfilt(sos, signal)
    return signal

In [6]:
names = os.listdir(root)
for name in tqdm(names):
    csv = pd.read_csv(os.path.join(root, name))
    signal = csv.drop('label', axis = 1).values
    signal = signal.reshape(-1)
    signal = prep(sos, signal)
    signal1 = signal.reshape(-1, 200)
    df = pd.DataFrame(signal1, columns = np.arange(200))
    df['label'] = csv['label']
    df.to_csv(os.path.join(root,name), index=False)

100%|███████████████████████████████████████| 1436/1436 [08:27<00:00,  2.83it/s]
